In [1]:
import sys
from paths import Paths
paths = Paths()
sys.path.append(paths.scripts_root)
import util


#### Waveform Variable Range Table

|Var  |BI |AA |AB  |AC  |AD   |AE  |AF  |WT   |CA  |HT  |CB  |PA  |CC   |PB   |CD  |PC   |CE   |PD   |CF  |
|-----|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|Val 1| 20|10 | -25| 15 | -6  |3.6 |2.4 |1    |0.4 |0.6 |1   |0.1 |0.5  |0.2  |0.5 |0.3  |1    |1    | 0.5|
|Val 2|30 |5  |-10 |10  |-3.6 |-5  |3.6 |0.1  |1   |1.5 |1.5 |0.3 |0.8  |0.7  |1   |1.2  |1.5  |1.5  | 1.2|
|Fxn  |-|-|- |- |-  |- |- |CBxAF|- |- |- |- |HTxAE|CAxPD|- |-  |BIxCD|AAxPC|- |



In [2]:
WF_var_ranges = {
"BI": [20  , 30  , []],
"AA": [10  , 5   , []],
"AB": [-25 , -10 , []],
"AC": [15  , 10  , []],
"AD": [-6  , -3.6, []],
"AE": [3.6 , -5  , []],
"AF": [2.4 , 3.6 , []],
"WT": [1   , 0.1 , ["CB","AF"]],
"CA": [0.4 , 1   , []],
"HT": [0.6 , 1.5 , []],
"CB": [1   , 1.5 , []],
"PA": [0.1 , 0.3 , []],
"CC": [0.5 , 0.8 , ["HT","AE"]],
"PB": [0.2 , 0.7 , ["CA","PD"]],
"CD": [0.5 , 1   , []],
"PC": [0.3 , 1.2 , []],
"CE": [1   , 1.5 , ["BI","CD"]],
"PD": [1   , 1.5 , ["AA","PC"]],
"CF": [0.5 , 1.2 , []]
}

In [6]:
WF_var_header_fields = []
WF_var_dependents = {}
WF_var_independents = {}
WF_var_state = {}

num_independents = 0

def initialize_WF_vars():
    global num_independents
    for k,v in WF_var_ranges.items():
        WF_var_header_fields.append(k)
        if len(v[2]) > 0:
            WF_var_dependents[k] = v
        else:
            WF_var_independents[k] = v
            num_independents += 1
            
#         WF_var_values[k] = WF_var_ranges[k][0]
        WF_var_state[k] = 0
        #print(f"{k}: {WF_var_ranges[k][0]}")


initialize_WF_vars()
num_fields = len(WF_var_header_fields)

print("Dependent vars:", WF_var_dependents)
print(f"Independent vars ({num_independents}): {WF_var_independents}")

Dependent vars: {'WT': [1, 0.1, ['CB', 'AF']], 'CC': [0.5, 0.8, ['HT', 'AE']], 'PB': [0.2, 0.7, ['CA', 'PD']], 'CE': [1, 1.5, ['BI', 'CD']], 'PD': [1, 1.5, ['AA', 'PC']]}
Independent vars (14): {'BI': [20, 30, []], 'AA': [10, 5, []], 'AB': [-25, -10, []], 'AC': [15, 10, []], 'AD': [-6, -3.6, []], 'AE': [3.6, -5, []], 'AF': [2.4, 3.6, []], 'CA': [0.4, 1, []], 'HT': [0.6, 1.5, []], 'CB': [1, 1.5, []], 'PA': [0.1, 0.3, []], 'CD': [0.5, 1, []], 'PC': [0.3, 1.2, []], 'CF': [0.5, 1.2, []]}


In [7]:
# '{0:b}'.format(42)

def toBinary(n, l):
    return ''.join(str(1 & int(n) >> i) for i in range(l)[::-1])

state = toBinary(19, 20)
print(state)

00000000000000010011


In [8]:
'''
    Calculates the state and values for each waveform parameter
    
    Before calling, these must already be initialized:
        WF_var_dependents  : dict of list of ranges for dependent variables
        WF_var_independents: dict of list of ranges for independent variables
        WF_var_state: dict of states (0 or 1) for each variable
    
    Function will fill out:
        WF_var_state: dict of states (0 or 1) for each variable
        WF_var_values: dict of values for each variable
        
    Returns:
        ANSI binary string of states 
    
'''
def calc_vals(iter, debug=False):
    if debug: 
        print("States Before...")
        for idx,(k,v) in enumerate(WF_var_state.items()):
            var_type = "d" if k in WF_var_dependents else "i"
            print(f"[{idx:02}] ({var_type}) {k}: {v}")

    if debug: print(f"Dependent States: {toBinary(iter, len(WF_var_independents))}\n")

    '''
        Calc the independent variables.
        
            'BI': [20, 30, []]
    '''
    for idx,(k,v) in enumerate(WF_var_independents.items()):
        state = 1 & iter >> idx
        WF_var_state[k] = state
        WF_var_values[k] = v[state]

    ''' 
        Calc the dependent variables.

            'WT': [1, 0.1, ['CB', 'AF']]
    '''
    for idx,(k,v) in enumerate(WF_var_dependents.items()):
        if debug: print(f"dependent var {k} is {v[2][0]} xor {v[2][1]} -> {WF_var_state[v[2][0]]} xor {WF_var_state[v[2][1]]} = {WF_var_state[v[2][0]] ^ WF_var_state[v[2][1]]}")
        state = WF_var_state[v[2][0]] ^ WF_var_state[v[2][1]]
        WF_var_state[k] = state
        WF_var_values[k] = v[state]

    if debug: print("States After...")
    state_lst = list(''.join('-' for i in range(num_fields*2)[::-1]))
    for idx,(k,v) in enumerate(WF_var_state.items()):
        if k in WF_var_independents:
            ansi_code = util.BLUE
            var_type = "i"
        else:
            ansi_code = util.RED
            var_type = "d"
        if debug: print(f"[{idx:02}] ({var_type}) {k}: {v}")
        state_lst[idx*2] = ansi_code
        state_lst[(idx*2)+1] = str(v)
        state_str = ''.join(state_lst)

    if debug: print("Values After...")
    for idx,(k,v) in enumerate(WF_var_values.items()):
        if k in WF_var_dependents:
            var_type = "d"
            vals = WF_var_dependents[k][:2]
        else:
            var_type = "i"
            vals = WF_var_independents[k][:2]
        if debug: print(f"[{idx:02}] ({var_type}) {k}: {v} (state: {WF_var_state[k]}, vals{vals})")

    return state_str



In [9]:
WF_var_values = {}

iter = 22020
state_str = calc_vals(iter, False)
print(state_str+util.RESET)
print(WF_var_values)

0010000100110001010
{'BI': 20, 'AA': 10, 'AB': -10, 'AC': 15, 'AD': -6, 'AE': 3.6, 'AF': 2.4, 'CA': 0.4, 'HT': 0.6, 'CB': 1.5, 'PA': 0.3, 'CD': 0.5, 'PC': 1.2, 'CF': 0.5, 'WT': 0.1, 'CC': 0.5, 'PB': 0.2, 'CE': 1, 'PD': 1.5}


In [10]:
cnt = pow(2,(len(WF_var_independents)))

s = set()
for iter in range(cnt):
    state_str = calc_vals(iter, False)
    #print(state_str+util.RESET)
    s.add(state_str)
print(f"cnt: {cnt}, set len: {len(s)}")

cnt: 16384, set len: 16384
